# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
import pandas as pd

df=pd.read_csv('homepage_actions.csv')

df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
df.info()

df["action"].value_counts

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


<bound method IndexOpsMixin.value_counts of 0       view
1       view
2       view
3       view
4       view
        ... 
8183    view
8184    view
8185    view
8186    view
8187    view
Name: action, Length: 8188, dtype: object>

In [3]:
# Users who viewed
viewed = df[df['action'] == 'view']['id'].unique()

# Users who clicked
clicked = df[df['action'] == 'click']['id'].unique()

# Users who both viewed and clicked
both_viewed_and_clicked = set(viewed) & set(clicked)

print(f"Number of users who both viewed and clicked: {len(both_viewed_and_clicked)}")


Number of users who both viewed and clicked: 1860


In [4]:
# Users who clicked but did not view
clicked_only = set(clicked) - set(viewed)

print(f"Number of users who clicked without viewing: {len(clicked_only)}")


Number of users who clicked without viewing: 0


In [5]:
# Check unique values in the group column
df['group'].value_counts()

# Cross-tabulate group by action
pd.crosstab(df['group'], df['action'])


action,click,view
group,,
control,932,3332
experiment,928,2996


In [6]:
# Check IDs by group
control_ids = df[df['group'] == 'control']['id'].unique()
experiment_ids = df[df['group'] == 'experiment']['id'].unique()

# Find any overlap
overlap_ids = set(control_ids) & set(experiment_ids)

print(f"Number of overlapping user IDs between control and experiment groups: {len(overlap_ids)}")


Number of overlapping user IDs between control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.


 : proportion of users who clicked in the control group.
​
 : proportion of users who clicked in the experiment group.

In [8]:
#Your code here
from statsmodels.stats.proportion import proportions_ztest

# Remove duplicate users if necessary
df = df.drop_duplicates(subset=['id', 'action'])

# Pivot data to get counts of views and clicks per group
clicks = df[df['action'] == 'click'].groupby('group')['id'].nunique()
views = df[df['action'] == 'view'].groupby('group')['id'].nunique()

# Number of clicks in each group
successes = [clicks['control'], clicks['experiment']]

# Number of total viewers in each group
nobs = [views['control'], views['experiment']]

# Run one-tailed z-test
stat, pval = proportions_ztest(successes, nobs, alternative='smaller')  # test if control < experiment
print(f"Z-statistic: {stat:.4f}")
print(f"P-value: {pval:.4f}")


Z-statistic: -2.6186
P-value: 0.0044


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [9]:
# Get views and clicks again
clicks = df[df['action'] == 'click'].groupby('group')['id'].nunique()
views = df[df['action'] == 'view'].groupby('group')['id'].nunique()

# Values we'll use
n_control = views['control']
n_experiment = views['experiment']

clicks_control = clicks['control']
clicks_experiment = clicks['experiment']

# Click-through rate for control group
p_control = clicks_control / n_control


In [10]:
#Your code here
expected_clicks_exp = n_experiment * p_control
print(f"Expected clicks in experiment group (under H0): {expected_clicks_exp:.2f}")


Expected clicks in experiment group (under H0): 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
#Your code here
# Standard deviation under H0
std_dev = (n_experiment * p_control * (1 - p_control)) ** 0.5

# Z-score
z_manual = (clicks_experiment - expected_clicks_exp) / std_dev
print(f"Z-score: {z_manual:.4f}")


Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
#Your code here
from scipy.stats import norm

# One-sided p-value (upper tail)
p_manual = 1 - norm.cdf(z_manual)
print(f"P-value: {p_manual:.4f}")


P-value: 0.0001


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.